# Creación de sistemas RAG sobre bases de datos vectoriales desde PDFs

Este proyecto implementa un sistema de Recuperación-Augmentación-Generación (RAG) que construye un almacén de vectores a partir del contenido de uno o varios archivos PDF. Utiliza el modelo `llama 3.2` junto con bibliotecas como `langchain` y `PyPDF2`.

## Instalación de dependencias

Antes de comenzar, instala las bibliotecas necesarias para ejecutar el proyecto:

```python
# Instalar librerías necesarias
# !pip install langchain PyPDF2 langchain_huggingface chromadb


# Extracción del contenido de un archivo PDF
Este código extrae el texto de un archivo PDF utilizando PyPDF2. Se recorre cada página del PDF y se concatena su contenido en una sola cadena de texto.

In [1]:
import PyPDF2
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

# División del texto en fragmentos
Para facilitar el procesamiento de grandes cantidades de texto, se divide el contenido en fragmentos manejables de tamaño fijo (500 caracteres en este caso).

In [2]:
def split_text(text, chunk_size=500):
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]


# Configuración del modelo LLM
Se utiliza el modelo llama 3.2 alojado en un servidor local para manejar las consultas.

In [3]:
llm = OllamaLLM(model="llama3.2", server_url="http://localhost:11434")


# Creación del modelo de embeddings y del almacén de vectores
El modelo de embeddings convierte el texto en representaciones vectoriales que se almacenan en una base de datos Chroma.

In [4]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = Chroma(persist_directory="./vectorstore", embedding_function=embedding_model)


/home/bigdata/miniconda3/envs/rag/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Procesamiento del contenido del PDF
Aquí se lee el contenido del archivo PDF, se divide en fragmentos y se imprime la cantidad total de fragmentos obtenidos.

In [5]:
pdf_path = "ciberseguridad.pdf"

page_content = extract_text_from_pdf(pdf_path)

chunks = split_text(page_content)
print(f"Number of chunks: {len(chunks)}")


Overwriting cache for 0 2994


Number of chunks: 84


# Adición de documentos al almacén de vectores
Cada fragmento de texto se encapsula en un objeto Document y se agrega al almacén de vectores para su posterior recuperación.

In [6]:
documents = [Document(page_content=chunk) for chunk in chunks]
vectorstore.add_documents(documents)
print("Documents added to the vector store.")


Documents added to the vector store.


# Definición del prompt para el sistema RAG
Se configura un prompt para guiar al modelo en la generación de respuestas basadas en el contenido recuperado del almacén de vectores.

In [7]:
prompt = ChatPromptTemplate.from_template(
    template="Usa el contexto a continuación para responder la pregunta del usuario:\n\n{context}\n\nPregunta: {question}\nRespuesta:"
)


# Configuración del sistema RAG
El sistema utiliza el almacén de vectores como recuperador y lo integra con el modelo LLM para formar una cadena de consulta y respuesta.

In [8]:
retriever = vectorstore.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)


# Consulta al sistema RAG
Finalmente, se realiza una consulta al sistema para obtener una respuesta basada en el contenido del archivo PDF.

In [10]:
query = "¿Cuál es el tema principal del documento?"
response = qa_chain.invoke(query)
print("Respuesta:", response)


Respuesta: {'query': '¿Cuál es el tema principal del documento?', 'result': 'El tema principal del documento es la Gestión del Riesgo de Seguridad de la Información, incluyendo los pasos y principios para identificar, priorizar y tratar riesgos en la seguridad de la información.'}
